In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import os

In [ ]:
ID_COOKIES_BUTTON = "onetrust-accept-btn-handler"
CLASS_ITEM_SALE = "new-item-box__overlay"
KEY_WORD_LOCATION = "Sverige"
DATA_TESTID_LOCATION_PARENT = "item-details-location"
CLASS_LOCATION_VALUE = "details-list__item-value"
CLASS_NEXT_PAGE = "web_ui__Pagination__next"
DIRECTORY_OUTPUT = "output"
FILENAME_OUTPUT_LOCATION = "location.json"
FILENAME_CONFIG = "config.txt"

PATH_OUTPUT_LOCATION = os.path.join(DIRECTORY_OUTPUT, FILENAME_OUTPUT_LOCATION)

In [ ]:
driver = webdriver.Chrome()

 Dans cette partie, on va accéder au site de Vinted. Si l'utilisateur a déjà effectué une recherche auparavant, nous souhaitons charger les filtres qu'il avait précédemment sélectionnés. Ces filtres sont stockés dans l'URL de la recherche précédente. S'il n'avait pas effectué de recherche, l'url de vinted par défaut est utilisée.

In [ ]:
url_vinted = "https://www.vinted.se/"

if os.path.isfile(FILENAME_CONFIG):
    with open(FILENAME_CONFIG, "r") as file:
        url_vinted = file.read()
    print("URL récupérée :", url_vinted)

In [ ]:
driver.get(url_vinted)

Accepter les cookies si nécessaire

In [ ]:
try:
    cookies_button = driver.find_element(By.ID, ID_COOKIES_BUTTON)
    cookies_button.click()
except Exception as e:
    # S'il n'y a pas la popup des cookies
    pass 

L'utilisateur doit d'abord choisir les filtres par défaut manuellement sur l'interface du navigateur. 

Une fois cette étape accomplie, il doit exécuter la cellule suivante pour enregistrer les filtres de Vinted afin de les réutiliser lors de ses prochaines visites.

In [ ]:
with open(FILENAME_CONFIG, "w") as file:
    file.write(driver.current_url)

On cherche maintenant à récupérer tous les URLS de tous les produits

In [ ]:
def extract_urls_page():
    """
    Extract all URLs of products for sale on the opened page.
    """
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.find_all("a", class_=CLASS_ITEM_SALE)
    urls = [item['href'] for item in items]
    return urls

On définit le dictionnaire qui va stocker les différentes url trié par localisation.

In [ ]:
urls_location = {}

On récupère les données déjà collectées lors des précédentes sessions.

In [ ]:
if os.path.exists(PATH_OUTPUT_LOCATION):
    with open(PATH_OUTPUT_LOCATION, "r") as file:
        urls_location = json.load(file)
    print("Données des recherches antérieures chargées")
else:
    print("Aucune recherche antérieure.")

On trie maintenant les urls en fonction de la localisation

In [ ]:
def sort_urls_by_location(urls_items):
    global urls_location

    url_product_list = driver.current_url

    for url in urls_items:
        driver.get(url)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        div_location = soup.find("div", {"data-testid": DATA_TESTID_LOCATION_PARENT})

        if div_location:
            location = div_location.find("div", class_=CLASS_LOCATION_VALUE)
            if location:
                if location.text in urls_location:
                    urls_location[location.text].append(url)
                else:
                    urls_location[location.text] = [url]
            else:
                print("Localisation non mentionnée")
        else:
            print("Description localisation manquante")
            print(driver.page_source)

        # driver.back()  
    driver.get(url_product_list) # Revenir en arrière pour continuer la recherche

Parcourir toute les pages

In [ ]:
def go_to_next_page():
    # Passer à la page suivante si elle existe
    try:
        next_button = driver.find_element(By.CLASS_NAME, CLASS_NEXT_PAGE)
        next_button.click()
        WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        return True
    except Exception as e:
        return False

In [ ]:
def remove_url_already_visited(urls_to_check, old_urls):
    res = []
    
    for url in urls_to_check:
        if url in old_urls:
            print(True, res)
            return True, res
        res.append(url)
    print(False, res)
    return False, res

In [ ]:
def navigate_between_pages():
    global urls_location

    urls_already_visited = []
    for location in urls_location.values():
        urls_already_visited.extend(location)

    next_page = True
    i = 0
    while next_page:
        print(i)
        i+=1
        
        urls_items = extract_urls_page()
        has_removed_url, urls_items_not_visited = remove_url_already_visited(urls_items, urls_already_visited)
        sort_urls_by_location(urls_items_not_visited)

        if has_removed_url: # On a atteint un item déjà visité, donc inutile de continuer
            next_page = False
        else:
            next_page = go_to_next_page()

In [ ]:
navigate_between_pages()

Filtrer les produits en fonction d'une localisation précise

In [ ]:
def filter_urls_by_location(location):
    global urls_location
    url_filtered = {}
    for key, value in urls_location.items():
        if location in key:
            url_filtered[key] = value
    return url_filtered
    

In [ ]:
filter_urls_by_location(KEY_WORD_LOCATION)

Sauvegarde des résultats

In [ ]:
if not os.path.exists(DIRECTORY_OUTPUT):
    os.makedirs(DIRECTORY_OUTPUT)

In [ ]:
with open(PATH_OUTPUT_LOCATION, "w") as file:
    json.dump(urls_location, file)

Fermer le navigateur

In [ ]:
driver.quit()